In [5]:
import findspark
import pandas as pd
findspark.init()

from pyspark.sql import SparkSession
from pyspark import SparkConf

# for shared metastore (shared across all users)
spark = SparkSession.builder.appName("Building dataset").config("hive.metastore.uris", "thrift://bialobog:9083", conf=SparkConf()).getOrCreate() \

# for local metastore (your private, invidivual database) add the following config to spark session
spark.sql("USE 2023_04_01")

DataFrame[]

In [50]:
import pyspark.pandas as ps

def get_all_stocks():
    query = f"""SELECT s.ticker_region, sc.fref_listing_exchange FROM sym_ticker_region s 
                LEFT JOIN FF_SEC_COVERAGE c ON c.fsym_id = s.fsym_id
                LEFT JOIN sym_coverage sc ON sc.fsym_id = s.fsym_id
                WHERE s.ticker_region LIKE "%-US" AND s.ticker_region NOT LIKE '%.%' AND c.CURRENCY = "USD"
                AND (sc.fref_listing_exchange = "NAS" OR sc.fref_listing_exchange = "NYS")"""
    df = spark.sql(query)
    df = df.withColumn("ticker_region", regexp_replace("ticker_region", "-US$", ""))
    ticker_list = [row.ticker_region for row in df.collect()]
    return ticker_list

def get_stocks_query():
    query = f"""SELECT s.ticker_region, s.fsym_id FROM sym_ticker_region s 
                LEFT JOIN FF_SEC_COVERAGE c ON c.fsym_id = s.fsym_id
                LEFT JOIN sym_coverage sc ON sc.fsym_id = s.fsym_id
                WHERE s.ticker_region LIKE "%-US" AND s.ticker_region NOT LIKE '%.%' AND c.CURRENCY = "USD"
                AND (sc.fref_listing_exchange = "NAS" OR sc.fref_listing_exchange = "NYS")"""
    df = spark.sql(query)
    print("stocks obtained")
    return df


def get_implosion_df(filename):
    df = pd.read_csv(filename, index_col=False)
    df['Implosion_Date'] = pd.to_datetime(df['Implosion_Date'])
    return df

def get_features_for_imploded_stocks(df):
    df=spark.createDataFrame(df)
    df.createOrReplaceTempView("temp_table")
    query = """SELECT t.Ticker, t.Implosion_Date, t.Implosion_Next_Year, a.date, a.ff_gross_inc, b.date, b.ff_gross_inc, c.date, c.ff_gross_inc
                FROM temp_table t 
                LEFT JOIN sym_ticker_region s ON s.ticker_region = CONCAT(t.Ticker, '-US')
                LEFT JOIN FF_BASIC_AF a ON s.fsym_id = a.fsym_id AND YEAR(a.date) = YEAR(t.Implosion_Date)-1
                LEFT JOIN FF_BASIC_AF b ON s.fsym_id = b.fsym_id AND YEAR(b.date) = YEAR(t.Implosion_Date)-2
                LEFT JOIN FF_BASIC_AF c ON s.fsym_id = c.fsym_id AND YEAR(c.date) = YEAR(t.Implosion_Date)-3
                ORDER BY t.Ticker, a.date
    """
    df = spark.sql(query)
    print(df.show(10))
    
    
def get_features_for_non_imploded(metric_string, metric_string2):
    df = get_stocks_query()
    df.createOrReplaceTempView("temp_table")
    query = f"""WITH RankedData AS (
    SELECT
        t.ticker_region,
        a.date,
        {metric_string},
        ROW_NUMBER() OVER (PARTITION BY t.ticker_region ORDER BY a.date DESC) AS row_num
    FROM
        temp_table t
        LEFT JOIN FF_ADVANCED_AF a ON a.fsym_id = t.fsym_id
)
SELECT
    ticker_region,
    date,
    {metric_string2}
FROM
    RankedData r
WHERE
    row_num <= 4
ORDER BY
    ticker_region,
    date"""
    new_df = spark.sql(query)
    return new_df

def create_non_imploded_ds():
    df_metrics = ps.DataFrame(spark.sql("SELECT * FROM FF_ADVANCED_AF LIMIT 10")) #get all the metrics
    cols = []
    for c in df_metrics.columns:
        if df_metrics[c].dtype=='float64':#get all the metrics we can calculate correlations with
            cols.append(c)
    metric_string = ', '.join('a.' + item for item in cols[:3])
    metric_string2 = ', '.join('r.' + item for item in cols[:3])
    df = ps.DataFrame(get_features_for_non_imploded(metric_string, metric_string2))
    df['Implosion_Next_Year'] = 0
    print(df.head(20))
    

def create_dataset():
    # df = get_implosion_df('imploded_stocks.csv')
    # df = df.drop(df.columns[0], axis=1)
    # df['Implosion_Year'] = df['Implosion_Date'].dt.year
    # df['Implosion_Next_Year'] = 1
    # get_features_for_imploded_stocks(df)
    #print(df.head())
    #df=spark.createDataFrame(df)
    #df.createOrReplaceTempView("temp_table")
    create_non_imploded_ds()

    
create_dataset()


stocks obtained
   ticker_region        date  ff_accr_exp_cf  ff_actg_chg_ps  ff_amort_cf  Implosion_Next_Year
0           A-US  2019-10-31            23.0             0.0   127.000000                    1
1           A-US  2020-10-31            29.0             0.0   189.000000                    1
2           A-US  2021-10-31           112.0             0.0   199.000000                    1
3           A-US  2022-10-31           -22.0             0.0   197.000000                    1
4          AA-US  2019-12-31          -175.0             0.0    19.000000                    1
5          AA-US  2020-12-31          -153.0             0.0     9.000000                    1
6          AA-US  2021-12-31           -38.0             0.0    11.000000                    1
7          AA-US  2022-12-31          -173.0             0.0     7.000000                    1
8        AAAP-US  2013-12-31             NaN             0.0     3.987880                    1
9        AAAP-US  2014-12-31      